<h1>
  <center>
    Whale Optimization Algorithm
  </center>
<h1>

## Problem
The optimization problem we choose to solve using the Whale Search Optimization algorithm consists in **function minimization**.

- **Goal**: Find global minimum $\min_ {x \in D} f(x)$  
- **Type**: Nonlinear Programming Problem (NLP)
- **Real-valued vector** $x \in \mathbb{R}^n$
- **Continuous, bounded domain**
- These problems are: **NP-hard**
- Often **multimodal, non-separable, non-convex**.

## Benchmarking in literature

Common practice since 1990s: test algorithms on diverse functions

Benchmark sets must include:
- Unimodal vs. Multimodal
- Separable vs. Non-separable
- Low- vs. High-dimensional
  
Helps identify where an algorithm is effective.
  
**Ackley, Griewank, Schwefel** = standard tests for:
- Robustness
- Exploration/exploitation
- Convergence


## Functions analyzed
- **Schwefel**'s 2.21 Function  
    
  <img 
      style="display: block; 
            margin-left: auto;
            margin-right: auto;
            width: 70%;"
      src="https://al-roomi.org/multimedia/Unconstrained_Benchmarks/ndimensional/SchwefelFunction221/3D_SchwefelFunction221.jpg" 
      alt="Schwefel">
  </img>
    
  <img 
      style="display: block; 
            margin-left: auto;
            margin-right: auto;
            width: 40%;"
      src="static\Schwefel_2.21.svg" 
      alt="Schwefel">
  </img>
  
- **Ackley**'s function  
    
  <img 
      style="display: block; 
            margin-left: auto;
            margin-right: auto;
            width: 70%;"
      src="https://www.sfu.ca/~ssurjano/ackley.png" 
      alt="Ackley">
  </img>
    
  <img 
      style="display: block; 
            margin-left: auto;
            margin-right: auto;
            width:70%;"
      src="static\Ackley.svg" 
      alt="Ackley">
  </img>

- **Griewank**'s Function  
    
  <img 
      style="display: block; 
            margin-left: auto;
            margin-right: auto;
            width: 70%;"
      src="https://al-roomi.org/multimedia/Unconstrained_Benchmarks/ndimensional/GeneralizedGriewankFunction/3D_GeneralizedGriewankFunction.jpg" 
      alt="Griewank">
  </img>
    
  <img 
      style="display: block; 
            margin-left: auto;
            margin-right: auto;
            width: 70%;"
      src="https://www.sfu.ca/~ssurjano/griewank.png" 
      alt="Griewank">
  </img>
    
  <img 
      style="display: block; 
            margin-left: auto;
            margin-right: auto;
            width:50%;"
      src="static\Griewank.svg" 
      alt="Griewank">
  </img>

Image sources: 
- Surjanovic, S. & Bingham, D. (2013). Virtual Library of Simulation Experiments: Test Functions and Datasets. http://www.sfu.ca/~ssurjano.
- Ali R. Al-Roomi (2015). Unconstrained Single-Objective Benchmark Functions Repository [https://www.al-roomi.org/benchmarks/unconstrained]. Halifax, Nova Scotia, Canada: Dalhousie University, Electrical and Computer Engineering.

In [ ]:
import numpy as np
import math
import time

## Prerequisites

---

**Functions**

In [ ]:
def sphere(x):
    """
        Sphere function - a simple test function for optimization
        f(x) = sum(x_i^2)
        Global minimum: f(0,...,0) = 0
    """
    return np.sum(x**2)

def schwefel(x):
    """
        Schwefel 2.21 function
        f(x) = max(|xi|) for i=1,...,n
        Global minimum: f(0,...,0) = 0
        Search space: [-100, 100]
    """
    return np.max(np.abs(x))

def ackley(x, a=20, b=0.2, c=2*np.pi):
    """
        Ackley function
        f(x) = -a * exp(-b * sqrt(1/d * sum(x_i^2))) - exp(1/d * sum(cos(c*x_i))) + a + exp(1)
        where:
        a = 20, b = 0.2, c = 2π are standard values
        d is the dimension of x
        Global minimum: f(0,...,0) = 0
        Search space: [-32, 32]
    """
    d = len(x)
    sum_sq = np.sum(x**2)
    sum_cos = np.sum(np.cos(c * x))

    term1 = -a * np.exp(-b * np.sqrt(sum_sq/d))
    term2 = -np.exp(sum_cos/d)

    return term1 + term2 + a + np.exp(1)

def griewank(x):
    """
        Griewank function
        f(x) = 1 + (1/4000)*sum(x_i^2) - prod(cos(x_i/sqrt(i)))
        Global minimum: f(0,...,0) = 0
        Search space: [-600, 600]

        The function has many widespread local minima, which are regularly distributed.
        The product term makes the function non-separable.
    """
    sum_term = np.sum(x**2) / 4000

    indices = np.arange(1, len(x) + 1)
    prod_term = np.prod(np.cos(x / np.sqrt(indices)))

    return 1 + sum_term - prod_term

---
**Helpers**

In [ ]:
def fitness(x, func):
    """
        Calculates the fitness value for a solution using the given function
    """
    return func(x)

def init_pop(pop_size, nr_dim, bounds):
    """
        Initialize population with random solutions within the given bounds
    """
    return np.random.uniform(bounds[0], bounds[1], size=(pop_size, nr_dim))

## Whale Optimization Algorithm

The core inspiration of the algorithm is the whales' bubble-net feeding technique, where whales create spiral-shaped bubbles to trap prey.  

Researchers observed two key maneuvers:  
- Upward spirals: whales swim in a helix while releasing bubbles upward
- Double loops: complex movement patterns around the prey

WOA simulates this behavior through three key mechanisms:
- Encircling prey – models exploitation
- Spiral updating position – mimics helix movement
- Search for prey – models exploration

Algorithm overview:
    
### Phase 1: Encircling Prey
Whales assume that the current best solution is the prey, and all other whales update their positions to move toward it.

Equations:
- Calculate distance to best solution: `D = |C * X_best - X|`
  
- Update position: `X(t + 1) = X_best - A * D`

Parameters:
- X = current whale position
- X_best = position of the best whale (best solution found so far)
- A and C = coefficient vectors that influence movement
- D = distance between current whale and best whale
- t = current iteration

How A and C are calculated: `A = 2 * a * r - a`, `C = 2 * r`

Where:
- a decreases from 2 → 0 over time (controls exploration vs exploitation)
- r is a random number in [0, 1]

### Phase 2: Bubble-Net Attacking Method (Exploitation Phase)
Whales use two behaviors to simulate hunting prey in water:

1. Shrinking Encircling Mechanism
  As a → 0, the whale gets closer to the prey.  
  Smaller |A| (< 1) pulls whale near X_best.

2. Spiral Updating Position  
  ```python
    X(t + 1) = D * e^(b * l) * cos(2πl) + X_best
  ```

Where:
- D = |X_best - X|
- b = constant defining spiral shape
- l = random number in [–1, 1]
- e^(...) * cos(...) = models the spiral movement of whales around prey

Combined Model (probabilistic behavior)

- The whale chooses either of the two methods with 50% probability:  
```python
  X(t + 1) = {
    X_best - A * D                        if p < 0.5
    D * e^(b * l) * cos(2πl) + X_best     if p ≥ 0.5
  }
```

Where p is a random number in [0, 1].

### Phase 3: Search for Prey (Exploration Phase)
When whales explore instead of exploit, they move relative to a random whale:

```python
  D = |C * X_rand - X|
  X(t + 1) = X_rand - A * D
```

Here:
- X_rand = position of a randomly selected whale
- Used when |A| > 1 to move whales away from the best and encourage exploration

```python
Initialize whale population
Evaluate fitness and get X_best

while (not max iterations):
    for each whale:
        Update a, A, C, l, p
        if p < 0.5:
            if |A| < 1:
                use encircling update (toward X_best)
            else:
                use exploration update (toward random X)
        else:
            use spiral update (around X_best)
        Correct position if out of bounds
    Evaluate fitness
    Update X_best if needed
```

---
**Whale optimization algorithm**

In [ ]:
def woa(pop_size: int, spiral_shape_const: float, nr_dim: int, func, bounds: tuple, nr_iter: int):
    """
        Performs whale optimization algorithm to minimize the value of a given function func
        in:
        pop_size - int, population size
        spiral_shape_const - float, controls the tightness of the log spiral
        nr_dim - int, number of dimensions of the function input
        func - function to minimize
        bounds - tuple, bounds of the values of the function
        nr_iter - int, maximum number of iterations
    """
    curr_pop = init_pop(pop_size, nr_dim, bounds)
    a = 2

    # find the initial best solution
    global_best = min(curr_pop, key=lambda x : fitness(x, func))
    global_best_fitness = fitness(global_best, func)

    for i in range(nr_iter):
        # update a linearly from 2 to 0
        a -= 2 * i / nr_iter

        for j in range(pop_size):
            whale = curr_pop[j]

            # generate random numbers for encircling mechanism
            r1 = np.random.uniform(0, 1)
            r2 = np.random.uniform(0, 1)

            A = 2 * a * r1 - a # determines search area size
            C = 2 * r2 # random weight for best solution

            if np.random.uniform(0, 1) < 0.5:
                if abs(A) < 1: # exploitation: shrinking circle
                    D = np.abs(C * global_best - whale)
                    whale = global_best - A * D
                else: # exploration: random search
                    random_whale = curr_pop[np.random.randint(0, pop_size)]
                    D = np.abs(C * random_whale - whale)
                    whale = random_whale - A * D
            else: # exploitation: spiral update
                D = np.abs(global_best - whale)
                spiral_param = np.random.uniform(-1, 1)
                whale = D * math.exp(spiral_shape_const * spiral_param) * math.cos(2 * math.pi * spiral_param) + global_best

            # bound constraints
            whale = np.clip(whale, bounds[0], bounds[1])
            whale_fitness = fitness(whale, func)

            if whale_fitness < global_best_fitness:
                global_best = whale
                global_best_fitness = whale_fitness

            curr_pop[j] = whale

    return global_best, global_best_fitness

---
**Test**

In [ ]:
pop_size = 30
spiral_const = 1.0
dimensions = 2
bounds = (-5, 5)
iterations = 100

best_solution, best_fitness = woa(
    pop_size=pop_size,
    spiral_shape_const=spiral_const,
    nr_dim=dimensions,
    func=sphere,
    bounds=bounds,
    nr_iter=iterations
)
print(f"Best solution found for sphere: {best_solution}")
print(f"Best fitness value for sphere: {best_fitness}\n")

best_solution, best_fitness = woa(
    pop_size=pop_size,
    spiral_shape_const=spiral_const,
    nr_dim=dimensions,
    func=schwefel,
    bounds=bounds,
    nr_iter=iterations
)
print(f"Best solution found for schwefel: {best_solution}")
print(f"Best fitness value for schwefel: {best_fitness}\n")

best_solution, best_fitness = woa(
    pop_size=pop_size,
    spiral_shape_const=spiral_const,
    nr_dim=dimensions,
    func=ackley,
    bounds=bounds,
    nr_iter=iterations
)
print(f"Best solution found for ackley: {best_solution}")
print(f"Best fitness value for ackley: {best_fitness}\n")

best_solution, best_fitness = woa(
    pop_size=pop_size,
    spiral_shape_const=spiral_const,
    nr_dim=dimensions,
    func=griewank,
    bounds=bounds,
    nr_iter=iterations
)
print(f"Best solution found for griewank: {best_solution}")
print(f"Best fitness value for griewank: {best_fitness}\n")

Best solution found for sphere: [-4.14824280e-08  5.39559512e-08]
Best fitness value for sphere: 4.632036496348162e-15

Best solution found for schwefel: [1.17737638e-06 4.66314034e-06]
Best fitness value for schwefel: 4.663140341106267e-06

Best solution found for ackley: [-5.72846948e-08 -4.88427938e-09]
Best fitness value for ackley: 1.6261355684221712e-07

Best solution found for griewank: [-0.00082983  0.00167137]
Best fitness value for griewank: 1.043546454582156e-06



##  Improved Whale Optimization Algorithm

### 1. *Why Improve WOA?*

The *Standard Whale Optimization Algorithm (WOA)* is inspired by how humpback whales hunt prey using bubble-net feeding. While effective, it suffers from:

* *Premature convergence*: it can get stuck in local optima.
* *Weak exploration*: it often fails to search new areas of the solution space effectively.

The *Improved WOA (IWOA)* was designed specifically to fix these issues.

### 2. *Key Differences Between IWOA and WOA*

### *Hybrid with Differential Evolution (DE)*

* *IWOA combines WOA with the mutation operator from Differential Evolution (DE)*.
* DE is strong at exploring the search space, so this combination enhances WOA’s exploration capabilities.
* Standard WOA does not use DE—it’s purely inspired by whale behavior.

### *Hybrid Search Operator*

* IWOA uses a *hybrid operator* that merges:

  * DE’s mutation
  * WOA’s encircling, prey-searching, and spiral behaviors
* It separates the algorithm into two distinct parts:

  * *Exploration part*: uses DE mutation + WOA's prey search
  * *Exploitation part*: uses WOA’s shrinking circle and spiral update

### **Adaptive Exploration–Exploitation Balance (with k)**

* IWOA introduces a new parameter k (defined in Eq. 8 in the paper).
* k starts at *1* and *gradually decreases to 0* over time.
* This lets the algorithm *explore more in early iterations* and *exploit more in later ones*.
* Standard WOA also switches between exploration/exploitation (using A and random p), but IWOA’s k-based control is more *systematic and adaptive*.

### *Elitism*

* IWOA is *elitist*, meaning it always keeps the best solution between:

  * the current whale position Xi, and
  * the new candidate position Ui from the DE-based mutation.
* In standard WOA, new positions replace the old ones directly without comparing fitness.

### *Selection Method*

* IWOA uses a *selection strategy similar to Differential Evolution*, helping it maintain diversity and choose better offspring.

### *Performance*

* In experiments, *IWOA outperforms WOA, especially on **complex, multimodal test functions* (those with many local optima).
* It finds better solutions but might take slightly more time to converge due to the deeper exploration.

## IWOA vs WOA - Summary table


| Feature                        | WOA (Standard)   | IWOA (Improved)                  |
| ------------------------------ | ---------------- | -------------------------------- |
| Exploration capability         | Weak             | Strong (via DE mutation)         |
| Premature convergence          | Possible         | Reduced                          |
| Hybrid with DE                 | No             | Yes                            |
| Adaptive balance control       | Basic (A, p) | Advanced (via decreasing k)    |
| Elitism                        | Not guaranteed | Always selects better solution |
| Selection method               | WOA-style        | DE-style                         |
| Performance on multimodal      | Moderate         | High                             |
| Spiral + encircling behavior   | Yes            | Yes                            |
| Random agent-based exploration | Yes            | Enhanced with DE mutation      |
  

Think of WOA as a group of whales hunting based on instinct. They might succeed, but they sometimes get confused or swim in circles.

Now imagine IWOA as whales that:

* Still use instinct,
* But also share smart strategies from other sea creatures (DE),
* Learn over time when to explore and when to focus,
* And always keep the best fish they catch.

---
**Improved whale optimization algorithm**

In [ ]:
def iwoa(pop_size: int, nr_dim: int, func, bounds: tuple, nr_iter: int, F: float = 0.5, CR: float = 0.7, bl: float = 1.0):
    """
        Improved Whale Optimization Algorithm (IWOA)

        pop_size : int, population size (number of whales)
        nr_dim : int, number of dimensions
        func : callable, objective function to minimize
        bounds : tuple, (lower_bound, upper_bound) for solution space
        nr_iter : int, maximum number of iterations
        F : float, differential weight (mutation factor) for DE operator
        CR : float, crossover probability for DE operator
        bl : float, spiral shape constant
    """
    # initialize population
    population = init_pop(pop_size, nr_dim, bounds)
    fitness_values = np.array([fitness(x, func) for x in population])

    # find the initial best solution
    best_idx = np.argmin(fitness_values)
    X_star = population[best_idx].copy()
    best_fitness = fitness_values[best_idx]

    t = 0  # current iteration
    while t < nr_iter:
        # calculate adaptive parameters
        lambda_t = 1 - (t/nr_iter)  # exploration-exploitation balance
        a = 2 - t * (2/nr_iter)     # linearly decreased from 2 to 0

        for i in range(pop_size):
            # Select three random indices different from i for DE
            r1, r2, r3 = np.random.choice([j for j in range(pop_size) if j != i], 3, replace=False)

            # Random number for probability check
            p = np.random.uniform(0, 1)

            # Random index for DE
            j_rand = np.random.randint(0, nr_dim)

            # Initialize offspring
            U = np.zeros(nr_dim)

            if p <= lambda_t:  # Exploration phase
                for j in range(nr_dim):
                    if np.random.uniform(0, 1) <= CR or j == j_rand:
                        # DE mutation
                        U[j] = X_star[j] + F * (population[r2][j] - population[r3][j])
                    else:
                        # Select random whale and calculate coefficients
                        x_rand = population[np.random.randint(0, pop_size)]
                        C = 2 * np.random.uniform(0, 1)  # Random coefficient
                        A = 2 * a * np.random.uniform(0, 1) - a  # Coefficient for search area
                        D = abs(C * x_rand[j] - population[i][j])
                        U[j] = x_rand[j] - A * D
            else:  # Exploitation phase
                if np.random.uniform(0, 1) <= 0.5:
                    # Calculate coefficients
                    C = 2 * np.random.uniform(0, 1)
                    A = 2 * a * np.random.uniform(0, 1) - a
                    # Calculate D using coefficient C
                    D = abs(C * X_star - population[i])
                    # Update position
                    U = X_star - A * D
                else:
                    # Spiral updating
                    D = abs(X_star - population[i])
                    l = np.random.uniform(-1, 1)
                    U = D * np.exp(bl * l) * np.cos(2 * np.pi * l) + X_star

            # Bound constraints
            U = np.clip(U, bounds[0], bounds[1])
            U_fitness = fitness(U, func)

            # Selection: elitism
            if U_fitness < fitness_values[i]:
                population[i] = U
                fitness_values[i] = U_fitness

                if U_fitness < best_fitness:
                    X_star = U.copy()
                    best_fitness = U_fitness

        t += 1

    return X_star, best_fitness

---
**Test**

In [ ]:
pop_size = 30
spiral_const = 1.0
dimensions = 2
bounds = (-5, 5)
iterations = 100
F = 0.5
CR = 0.7

best_solution, best_fitness = iwoa(
    pop_size=pop_size,
    nr_dim=dimensions,
    func=sphere,
    bounds=bounds,
    nr_iter=iterations,
    F=F,
    CR=CR
)
print(f"Best solution found for sphere: {best_solution}")
print(f"Best fitness value for sphere: {best_fitness}\n")

best_solution, best_fitness = iwoa(
    pop_size=pop_size,
    nr_dim=dimensions,
    func=schwefel,
    bounds=bounds,
    nr_iter=iterations,
    F=F,
    CR=CR
)
print(f"Best solution found for schwefel: {best_solution}")
print(f"Best fitness value for schwefel: {best_fitness}\n")

best_solution, best_fitness = iwoa(
    pop_size=pop_size,
    nr_dim=dimensions,
    func=ackley,
    bounds=bounds,
    nr_iter=iterations,
    F=F,
    CR=CR
)
print(f"Best solution found for ackley: {best_solution}")
print(f"Best fitness value for ackley: {best_fitness}\n")

best_solution, best_fitness = iwoa(
    pop_size=pop_size,
    nr_dim=dimensions,
    func=griewank,
    bounds=bounds,
    nr_iter=iterations,
    F=F,
    CR=CR
)
print(f"Best solution found for griewank: {best_solution}")
print(f"Best fitness value for griewank: {best_fitness}\n")

Best solution found for sphere: [-6.02351452e-35  5.98405124e-35]
Best fitness value for sphere: 7.209159646328645e-69

Best solution found for schwefel: [-2.66844621e-25  2.66900408e-25]
Best fitness value for schwefel: 2.66900407502994e-25

Best solution found for ackley: [-3.12454354e-16 -3.62972347e-17]
Best fitness value for ackley: 4.440892098500626e-16

Best solution found for griewank: [-2.11312767e-09  1.48973889e-08]
Best fitness value for griewank: 0.0



## Papers Analyzed
1. Seyedali Mirjalili, Andrew Lewis, The Whale Optimization Algorithm, Advances in Engineering Software, Volume 95, 2016, Pages 51-67, ISSN 0965-9978, https://doi.org/10.1016/j.advengsoft.2016.01.008.
  
2. Xiang Wang, Liangsa Wang, Han Li, Yibin Guo, An Improved Whale Optimization Algorithm for Global Optimization and Realized Volatility Prediction, Computers, Materials and Continua, Volume 77, Issue 3, 2023, Pages 2935-2969, ISSN 1546-2218, https://doi.org/10.32604/cmc.2023.044948.

## Results for the selected functions
### 1. WOA - Seyedali Mirjalili, Andrew Lewis
WOA algorithm performance compared with PSO, GSA, DE.  
  
| F   | WOA ave     | WOA std   | PSO ave    | PSO std   | GSA ave    | GSA std   | DE ave     | DE std    |
|-----|-------------|-----------|------------|-----------|------------|-----------|------------|-----------|
| F4 - Schwefel 2.21 | 0.072581    | 0.39747   | 1.086481   | 0.317039  | 7.35487    | 1.741452  | 0          | 0         |
| F10 - Ackley | 7.4043      | 9.897572  | 0.276015   | 0.50901   | 0.062087   | 0.23628   | 9.7E−08    | 4.2E−08   |
| F11 - Griewank | 0.000289    | 0.001586  | 0.009215   | 0.007724  | 27.70154   | 5.040343  | 0          | 0         |

### 2. IWOA - Xiang Wang, Liangsa Wang, Han Li, Yibin Guo
IWOA algorithm performance compared with DE and WOA.
  
| Fun. | DE mean  | DE Std.  | WOA mean  | WOA Std.  | IWOA mean | IWOA Std. |
|------|----------|----------|-------------|-------------|-----------|-----------|
| F4 - Schwefel 2.21 | 0.0011   | 0.0014   | 34.517    | 27.501    | 1.13e−4   | 1.38e−4   |
| F7 - Ackley | 1.9390   | 1.1466   | 7.93e−9   | 1.40e−9   | 8.85e−9   | 8.2e−10   |
| F9 - Griewank | 0.0162   | 0.0200   | 0.0018    | 0.0046    | 0.0027    | 0.0065    |

## WOA Experiments

---

**Functions**

In [ ]:
def woa_n_times(n, pop_size, spiral_shape_const, nr_dim, func, bounds, nr_iter):
  """
    Runs the Whale Optimization Algorithm n times and returns the obtained results as a list
  """
  best_solutions = []
  for step in range(n):
    _, best_fitness = woa(pop_size=pop_size, spiral_shape_const=spiral_const, nr_dim=nr_dim, func=func, bounds=bounds, nr_iter=nr_iter)
    best_solutions.append(best_fitness)
  return best_solutions

---

**Helpers**

In [ ]:
def lists_to_markdown_table(header, *lists):
  """
    Returns a string formatted like a markdown table which contains data from the header and the lists
  """
  markdown_table = header
  n = len(lists[0])
  for i in range(n):
    markdown_table += "|"
    for list in lists:
      markdown_table += f" {list[i]} |"
    markdown_table += "\n"
  return markdown_table

---

**WOA ideal parameters**  

We have several parameters which affect the performance of the Whale Optimization algorithm:  
- Population size: `pop_size`
- Number of iterations: `nr_iter`
- Shape of the logarithmic spiral: `spiral_shape_const`

Following the works of *Seyedali Mirjalili and Andrew Lewis*, let us consider their proposed parameters:
```python
  pop_size = 30
  nr_iter = 500
  spiral_shape_const = 1.0
```
and tweak these values to see if we can determine a better performing configuration.  

For the sake of minimizing computational resources, the following comparative analysis will be performed on all three functions, considered on a low dimensionality setting:
```python
  func in [schwefel, ackley, griewank]
  nr_dim = 2
```

In [ ]:
# Test the algorithm for different values for constants

func_values = [schwefel, ackley, griewank]
bounds_values = [(-100, 100), (-32, 32), (-600, 600)]

pop_size_values = [10, 20, 30, 50]
nr_iter_values = [50, 100, 500]
spiral_shape_const_values = [0.8, 1.0, 1.2]
# total expermients: 3 * 4 * 3 * 3 = 108

func_list = []
pop_size_list = []
nr_iter_list = []
spiral_shape_const_list = []

fitness_list = []
execution_time_list = []

for func, bounds in zip(func_values, bounds_values):
  for pop_size in pop_size_values:
    for nr_iter in nr_iter_values:
      for spiral_shape_const in spiral_shape_const_values:
        # perform 30 identical experiments
        start_time = time.time()
        best_fitnesses = woa_n_times(30, pop_size, spiral_shape_const, nr_dim=2, func=func, bounds=bounds, nr_iter=nr_iter)
        end_time = time.time()

        average_fitness = np.array(best_fitnesses).mean()
        average_execution_time = (end_time - start_time) / 30

        func_list.append(func.__name__)
        pop_size_list.append(pop_size)
        nr_iter_list.append(nr_iter)
        spiral_shape_const_list.append(spiral_shape_const)

        fitness_list.append(average_fitness)
        execution_time_list.append(average_execution_time)

# Display experiment results in markdown table
header = "| Function | Population size | Maximum iterations number | Shape of the logarithmic spiral | Average fitness | Average execution time |\n"
header += "|---|---|---|---|---|---|\n"
markdown_table = lists_to_markdown_table(header, func_list, pop_size_list, nr_iter_list, spiral_shape_const_list, fitness_list, execution_time_list)

---

**Schwefel**  

| Function | Population size | Maximum iterations number | Shape of the logarithmic spiral | Average fitness | Average execution time |
|---|---|---|---|---|---|
| schwefel | 10 | 50 | 0.8 | 2.1147749439419914 | 0.01084757645924886 |
| schwefel | 10 | 50 | 1.0 | 2.169193472798216 | 0.010425217946370443 |
| schwefel | 10 | 50 | 1.2 | 2.8722862308369153 | 0.010216943422953288 |
| schwefel | 10 | 100 | 0.8 | 1.2551760657134745 | 0.021474313735961915 |
| schwefel | 10 | 100 | 1.0 | 1.4957937250951592 | 0.02076102097829183 |
| schwefel | 10 | 100 | 1.2 | 2.4928800182250153 | 0.021450535456339518 |
| schwefel | 10 | 500 | 0.8 | 0.07637026594324388 | 0.10627922217051188 |
| schwefel | 10 | 500 | 1.0 | 0.0867041614306447 | 0.13146949609120687 |
| schwefel | 10 | 500 | 1.2 | 0.16386569343091037 | 0.11233092943827311 |
| schwefel | 20 | 50 | 0.8 | 0.5640390734663644 | 0.021155007680257163 |
| schwefel | 20 | 50 | 1.0 | 1.1553386781723476 | 0.020427759488423666 |
| schwefel | 20 | 50 | 1.2 | 1.310555245787202 | 0.021529014905293783 |
| schwefel | 20 | 100 | 0.8 | 0.6930840519045334 | 0.043979811668396 |
| schwefel | 20 | 100 | 1.0 | 0.40325596011385634 | 0.04196035861968994 |
| schwefel | 20 | 100 | 1.2 | 0.3472276669929945 | 0.042486961682637533 |
| schwefel | 20 | 500 | 0.8 | 0.032006123948627806 | 0.24379413922627766 |
| schwefel | 20 | 500 | 1.0 | 0.06777666509309689 | 0.21683497428894044 |
| schwefel | 20 | 500 | 1.2 | 0.06336136218832347 | 0.2404498815536499 |
| schwefel | 30 | 50 | 0.8 | 0.13562556008040913 | 0.03158589998881022 |
| schwefel | 30 | 50 | 1.0 | 0.5985966079317172 | 0.03177532354990641 |
| schwefel | 30 | 50 | 1.2 | 0.5269516142539987 | 0.03162946701049805 |
| schwefel | 30 | 100 | 0.8 | 0.15428818822246143 | 0.06584192117055257 |
| schwefel | 30 | 100 | 1.0 | 0.2625416840151219 | 0.09138884544372558 |
| schwefel | 30 | 100 | 1.2 | 0.19340438844103397 | 0.06308844089508056 |
| schwefel | 30 | 500 | 0.8 | 0.020431710132645966 | 0.35080628395080565 |
| schwefel | 30 | 500 | 1.0 | 0.02241861669777239 | 0.34746580918629966 |
| schwefel | 30 | 500 | 1.2 | 0.0069945266782915775 | 0.3313821236292521 |
| schwefel | 50 | 50 | 0.8 | 0.19644636443332622 | 0.07270329793294271 |
| schwefel | 50 | 50 | 1.0 | 0.21554655721426305 | 0.06408242384592693 |
| schwefel | 50 | 50 | 1.2 | 0.08006850170090693 | 0.052068289120992026 |
| schwefel | 50 | 100 | 0.8 | 0.03524187888257878 | 0.10505839983622232 |
| schwefel | 50 | 100 | 1.0 | 0.09458680342095085 | 0.10571972529093425 |
| schwefel | 50 | 100 | 1.2 | 0.050774417083984544 | 0.1354568084081014 |
| schwefel | 50 | 500 | 0.8 | 0.01513470003600456 | 0.5654570420583089 |
| schwefel | 50 | 500 | 1.0 | 0.0008357506334215771 | 0.5729062716166179 |
| schwefel | 50 | 500 | 1.2 | 0.006358808209472549 | 0.5869913895924886 |

---

**Ackley**  

| Function | Population size | Maximum iterations number | Shape of the logarithmic spiral | Average fitness | Average execution time |
|---|---|---|---|---|---|
| ackley | 10 | 50 | 0.8 | 0.7279773506236356 | 0.01627781391143799 |
| ackley | 10 | 50 | 1.0 | 0.6900699343776105 | 0.01599275271097819 |
| ackley | 10 | 50 | 1.2 | 0.6426992135124007 | 0.016341026624043783 |
| ackley | 10 | 100 | 0.8 | 0.199974166952098 | 0.03227355480194092 |
| ackley | 10 | 100 | 1.0 | 0.4978502991677164 | 0.031152629852294923 |
| ackley | 10 | 100 | 1.2 | 0.5787833546572109 | 0.032844710350036624 |
| ackley | 10 | 500 | 0.8 | 0.0004313722540373692 | 0.1941827138264974 |
| ackley | 10 | 500 | 1.0 | 0.08766110525889023 | 0.16004429658253988 |
| ackley | 10 | 500 | 1.2 | 0.03532814532348718 | 0.1924646536509196 |
| ackley | 20 | 50 | 0.8 | 0.2579022096761928 | 0.03202908833821615 |
| ackley | 20 | 50 | 1.0 | 0.1820057562859746 | 0.031733258565266924 |
| ackley | 20 | 50 | 1.2 | 0.2416934828235765 | 0.03220458825429281 |
| ackley | 20 | 100 | 0.8 | 0.012172895226550857 | 0.06656161944071452 |
| ackley | 20 | 100 | 1.0 | 0.044346580374318115 | 0.0634470542271932 |
| ackley | 20 | 100 | 1.2 | 0.02827651625485652 | 0.06401519775390625 |
| ackley | 20 | 500 | 0.8 | 4.8651534682401613e-08 | 0.35085386435190835 |
| ackley | 20 | 500 | 1.0 | 1.277767056606649e-08 | 0.35354061126708985 |
| ackley | 20 | 500 | 1.2 | 0.00223461229348129 | 0.35177884101867674 |
| ackley | 30 | 50 | 0.8 | 0.12847307055942028 | 0.04794496695200602 |
| ackley | 30 | 50 | 1.0 | 0.012405308572132364 | 0.04726948738098145 |
| ackley | 30 | 50 | 1.2 | 0.05196086573004021 | 0.04728470643361409 |
| ackley | 30 | 100 | 0.8 | 2.770631480778126e-05 | 0.12530383268992107 |
| ackley | 30 | 100 | 1.0 | 0.004883097314341208 | 0.09534982840220134 |
| ackley | 30 | 100 | 1.2 | 0.00017153070571926203 | 0.0946324348449707 |
| ackley | 30 | 500 | 0.8 | 9.217382412884945e-11 | 0.5068317969640096 |
| ackley | 30 | 500 | 1.0 | 6.761983565676626e-11 | 0.540308936436971 |
| ackley | 30 | 500 | 1.2 | 6.942754223378718e-10 | 0.5106717665990194 |
| ackley | 50 | 50 | 0.8 | 0.18474483584456688 | 0.07964417934417725 |
| ackley | 50 | 50 | 1.0 | 0.00018069173151700942 | 0.07911643981933594 |
| ackley | 50 | 50 | 1.2 | 0.00017919971584262246 | 0.09946710268656413 |
| ackley | 50 | 100 | 0.8 | 0.015283841487052256 | 0.1702066421508789 |
| ackley | 50 | 100 | 1.0 | 3.8809734291541295e-05 | 0.15846978823343913 |
| ackley | 50 | 100 | 1.2 | 0.0007408795183227271 | 0.19043423334757487 |
| ackley | 50 | 500 | 0.8 | 3.197146251447217e-13 | 0.8615201870600383 |
| ackley | 50 | 500 | 1.0 | 1.7760903858743405e-12 | 0.8586346069971721 |
| ackley | 50 | 500 | 1.2 | 4.611274325346433e-12 | 0.8561736027399699 |

---

**Griewank**  

| Function | Population size | Maximum iterations number | Shape of the logarithmic spiral | Average fitness | Average execution time |
|---|---|---|---|---|---|
| griewank | 10 | 50 | 0.8 | 0.18579673889714335 | 0.01515498956044515 |
| griewank | 10 | 50 | 1.0 | 0.18156228460663199 | 0.014385668436686198 |
| griewank | 10 | 50 | 1.2 | 0.271042605488646 | 0.014597185452779134 |
| griewank | 10 | 100 | 0.8 | 0.15681290830857303 | 0.02942354679107666 |
| griewank | 10 | 100 | 1.0 | 0.08293668249432122 | 0.03270517190297444 |
| griewank | 10 | 100 | 1.2 | 0.15280098002004897 | 0.04858164787292481 |
| griewank | 10 | 500 | 0.8 | 0.06093722812851679 | 0.15571196873982748 |
| griewank | 10 | 500 | 1.0 | 0.044032566500385055 | 0.14960734049479166 |
| griewank | 10 | 500 | 1.2 | 0.04459713638935182 | 0.17772446473439535 |
| griewank | 20 | 50 | 0.8 | 0.11855250501879597 | 0.028879459698994955 |
| griewank | 20 | 50 | 1.0 | 0.15259890654891517 | 0.02907880942026774 |
| griewank | 20 | 50 | 1.2 | 0.06266015265950448 | 0.029256288210550943 |
| griewank | 20 | 100 | 0.8 | 0.05312299254510878 | 0.05941455364227295 |
| griewank | 20 | 100 | 1.0 | 0.06498213654256632 | 0.058414586385091144 |
| griewank | 20 | 100 | 1.2 | 0.11467840526064732 | 0.058118430773417155 |
| griewank | 20 | 500 | 0.8 | 0.06640601283315131 | 0.32689066727956134 |
| griewank | 20 | 500 | 1.0 | 0.043225635269313355 | 0.32628889083862306 |
| griewank | 20 | 500 | 1.2 | 0.041142011442337784 | 0.3244289954503377 |
| griewank | 30 | 50 | 0.8 | 0.03717319366518149 | 0.044050200780232744 |
| griewank | 30 | 50 | 1.0 | 0.09305304457042128 | 0.04460682074228923 |
| griewank | 30 | 50 | 1.2 | 0.06178368339781661 | 0.04387011528015137 |
| griewank | 30 | 100 | 0.8 | 0.0668092836783757 | 0.08798308372497558 |
| griewank | 30 | 100 | 1.0 | 0.06679005680407046 | 0.1177813212076823 |
| griewank | 30 | 100 | 1.2 | 0.05713945245906417 | 0.08816705544789633 |
| griewank | 30 | 500 | 0.8 | 0.01844456650760068 | 0.47322351932525636 |
| griewank | 30 | 500 | 1.0 | 0.03963944608144166 | 0.4743857781092326 |
| griewank | 30 | 500 | 1.2 | 0.0253106769088336 | 0.4747759739557902 |
| griewank | 50 | 50 | 0.8 | 0.03498274198682183 | 0.10187543233235677 |
| griewank | 50 | 50 | 1.0 | 0.041162358107998044 | 0.07320716381072997 |
| griewank | 50 | 50 | 1.2 | 0.056415155672093806 | 0.07385179996490479 |
| griewank | 50 | 100 | 0.8 | 0.02737443416887671 | 0.14615946610768635 |
| griewank | 50 | 100 | 1.0 | 0.024987198359027147 | 0.17657848993937175 |
| griewank | 50 | 100 | 1.2 | 0.027517752808428244 | 0.14842166105906168 |
| griewank | 50 | 500 | 0.8 | 0.017859202146805406 | 0.8009604056676228 |
| griewank | 50 | 500 | 1.0 | 0.010888115012351974 | 0.7995774745941162 |
| griewank | 50 | 500 | 1.2 | 0.0073614535248358705 | 0.8011279741923014 |

---
**Analyzing individual parameter performance**

In [ ]:
# Analyse fitness by each parameter option

pop_size_values = [10, 20, 30, 50]
nr_iter_values = [50, 100, 500]
spiral_shape_const_values = [0.8, 1.0, 1.2]

# average fitness and average time by pop_size
print("Average fitness and average time by population size:")
for pop_size in pop_size_values:
  fitnesses = []
  execution_times = []
  for i in range(len(fitness_list)):
    if pop_size_list[i] == pop_size:
      fitnesses.append(fitness_list[i])
      execution_times.append(execution_time_list[i])
  average_fitness = np.array(fitnesses).mean()
  average_execution_time = np.array(execution_times).mean()
  # print results
  print(f"pop_size value: {pop_size}, average fitness: {average_fitness}, average execution time: {average_execution_time}")
print()

# average fitness and average time by nr_iter
print("Average fitness and average time by number of iterations:")
for nr_iter in nr_iter_values:
  fitnesses = []
  execution_times = []
  for i in range(len(fitness_list)):
    if nr_iter_list[i] == nr_iter:
      fitnesses.append(fitness_list[i])
      execution_times.append(execution_time_list[i])
  average_fitness = np.array(fitnesses).mean()
  average_execution_time = np.array(execution_times).mean()
  # print results
  print(f"nr_iter value: {nr_iter}, average fitness: {average_fitness}, average execution time: {average_execution_time}")
print()

# average fitness and average time by spiral_shape_const
print("Average fitness and average time by spiral shape constant:")
for spiral_shape_const in spiral_shape_const_values:
  fitnesses = []
  execution_times = []
  for i in range(len(fitness_list)):
    if spiral_shape_const_list[i] == spiral_shape_const:
      fitnesses.append(fitness_list[i])
      execution_times.append(execution_time_list[i])
  average_fitness = np.array(fitnesses).mean()
  average_execution_time = np.array(execution_times).mean()
  # print results
  print(f"spiral_shape_const value: {spiral_shape_const}, average fitness: {average_fitness}, average execution time: {average_execution_time}")
print()


Average fitness and average time by population size:
pop_size value: 10, average fitness: 0.643271801865788, average execution time: 0.06573042192576845
pop_size value: 20, average fitness: 0.22676465556117936, average execution time: 0.12887232274184993
pop_size value: 30, average fitness: 0.0957525140954366, average execution time: 0.192940953337116
pop_size value: 50, average fitness: 0.04239668338668326, average execution time: 0.3235507517685125

Average fitness and average time by number of iterations:
nr_iter value: 50, average fitness: 0.4543471705747371, average execution time: 0.03990459905730353
nr_iter value: 100, average fitness: 0.27102102247965437, average execution time: 0.08153076900376215
nr_iter value: 500, average fitness: 0.03077104812742384, average execution time: 0.4118854692688695

Average fitness and average time by spiral shape constant:
spiral_shape_const value: 0.8, average fitness: 0.21288550645202783, average execution time: 0.1775650130377876
spiral_shap

We see that, on average, the best values for the three parameters are the following:
```python
  pop_size = 50
  nr_iter = 500
  spiral_shape_const = 0.8
```
Now, let us extract the 10 actual best performing configurations in terms of average fitness.

---

**Top 10 best performing configurations**

In [ ]:
# Extract and print best 10 configurations

average_fitnesses_list = []
average_execution_times_list = []
values_no = (int) (len(fitness_list) / 3)

for i in range(values_no):
  average_fitness = (fitness_list[i] + fitness_list[values_no + i] + fitness_list[2 * values_no + i] ) / 3
  average_execution_time = (execution_time_list[i] + execution_time_list[values_no + i] + execution_time_list[2 * values_no + i] ) / 3
  average_fitnesses_list.append(average_fitness)
  average_execution_times_list.append(average_execution_time)

best_pop_size_list = []
best_nr_iter_list = []
best_spiral_shape_const_list = []

best_average_fitnesses_list = []
best_average_execution_times_list = []

best_indices = [average_fitnesses_list.index(x) for x in sorted(average_fitnesses_list, reverse=True)[-10:]]
best_indices.reverse()

for i in best_indices:
  best_pop_size_list.append(pop_size_list[i])
  best_nr_iter_list.append(nr_iter_list[i])
  best_spiral_shape_const_list.append(spiral_shape_const_list[i])
  best_average_fitnesses_list.append(average_fitnesses_list[i])
  best_average_execution_times_list.append(average_execution_times_list[i])

# Display experiment results in markdown table
header = "| No. | Population size | Maximum iterations number | Shape of the logarithmic spiral | Average fitness | Average execution time |\n"
header += "|---|---|---|---|---|---|\n"
markdown_table_2 = lists_to_markdown_table(header, list(range(1, 11)), best_pop_size_list, best_nr_iter_list, best_spiral_shape_const_list, best_average_fitnesses_list, best_average_execution_times_list)

---

| No. | Population size | Maximum iterations number | Shape of the logarithmic spiral | Average fitness | Average execution time |
|---|---|---|---|---|---|
| 1 | 50 | 500 | 1.0 | 0.00390795521584988 | 0.7437061177359686 |
| 2 | 50 | 500 | 1.2 | 0.004573420579639898 | 0.7480976555082534 |
| 3 | 30 | 500 | 1.2 | 0.010768401427133534 | 0.4389432880613539 |
| 4 | 50 | 500 | 0.8 | 0.010997967394376562 | 0.7426458782619901 |
| 5 | 30 | 500 | 0.8 | 0.012958758910806825 | 0.44362053341335717 |
| 6 | 30 | 500 | 1.0 | 0.02068602094894463 | 0.4540535079108344 |
| 7 | 50 | 100 | 0.8 | 0.02596671817950258 | 0.14047483603159586 |
| 8 | 50 | 100 | 1.2 | 0.026344349803578502 | 0.15810423427157932 |
| 9 | 20 | 500 | 0.8 | 0.0328040618111046 | 0.3071795569525824 |
| 10 | 20 | 500 | 1.2 | 0.03557932864138085 | 0.3055525726742214 |

We can extract the following observations:

- It seems that the most important two factors are `pop_size` and `nr_iter`. By grouping entries based on these two factors, we see both similar average fitnesses and average execution times.  

- We observe that the configuration proposed by *Seyedali Mirjalili and Andrew Lewis* is sixth in our list. It seems to be outperformed by two other configurations with similar parameters, configurations *3* and *5*.  

- Depending on computational resources available and desired accuracy, one can choose one of the following three configurations:
```python
  # Higher execution time, best fitness
  pop_size, nr_iter, spiral_shape_const = 50, 500, 1.0
  # Medium execution time, good fitness
  pop_size, nr_iter, spiral_shape_const = 30, 500, 1.2
  # Low execution time, acceptable fitness
  pop_size, nr_iter, spiral_shape_const = 50, 100, 0.8
```

Moving forward, let us use the *best performing configuration* found to test the scalability of our algorithm, by analyzing performance based on the problem dimension: `nr_dim`.

---

**Performance based on function dimensions**

In [ ]:
# Test the algorithm for different function dimensions

func_values = [schwefel, ackley, griewank]
bounds_values = [(-100, 100), (-32, 32), (-600, 600)]

nr_dim_values = [2, 5, 10, 20, 30]
# total expermients: 3 * 5 = 15

func_list = []
nr_dim_list = []

fitness_list = []
std_dev_list = []
execution_time_list = []

pop_size, nr_iter, spiral_shape_const = 50, 500, 1.0

for func, bounds in zip(func_values, bounds_values):
  for nr_dim in nr_dim_values:
    # perform 30 identical experiments
    start_time = time.time()
    best_fitnesses = woa_n_times(30, pop_size=pop_size, spiral_shape_const=spiral_shape_const, nr_dim=nr_dim, func=func, bounds=bounds, nr_iter=nr_iter)
    end_time = time.time()

    average_fitness = np.array(best_fitnesses).mean()
    average_execution_time = (end_time - start_time) / 30

    func_list.append(func.__name__)
    nr_dim_list.append(nr_dim)

    fitness_list.append(average_fitness)
    execution_time_list.append(average_execution_time)

# Display experiment results in markdown table
header = "| Function | Number of dimensions | Average fitness | Average execution time |\n"
header += "|---|---|---|---|\n"
markdown_table_3 = lists_to_markdown_table(header, func_list, nr_dim_list, fitness_list, execution_time_list)

---

**Schwefel**  

| Function | Number of dimensions | Average fitness | Average execution time |
|---|---|---|---|
| schwefel | 2 | 0.0037119000631061863 | 0.5726394335428874 |
| schwefel | 5 | 0.029398451621899937 | 0.6078199863433837 |
| schwefel | 10 | 0.06307347093037435 | 0.5758486827214558 |
| schwefel | 20 | 0.09019432445976096 | 0.5975637435913086 |
| schwefel | 30 | 0.15999849124116666 | 0.5674148241678874 |

---

**Ackley**  

| Function | Number of dimensions | Average fitness | Average execution time |
|---|---|---|---|
| ackley | 2 | 2.0400570122092177e-12 | 0.8528265396753947 |
| ackley | 5 | 4.5672992706376667e-08 | 0.8565802494684855 |
| ackley | 10 | 7.219194343062915e-07 | 0.8605231285095215 |
| ackley | 20 | 2.126236645919969e-06 | 0.8868021885553996 |
| ackley | 30 | 3.926561614253643e-06 | 0.9013636906941732 |

---

**Griewank**  

| Function | Number of dimensions | Average fitness | Average execution time |
|---|---|---|---|
| griewank | 2 | 0.01778917237602095 | 0.7901161988576253 |
| griewank | 5 | 0.13679965481720885 | 1.185421888033549 |
| griewank | 10 | 0.10582387823752236 | 1.1840367873509725 |
| griewank | 20 | 0.02741013785429284 | 1.1821806033452351 |
| griewank | 30 | 0.032910973861624074 | 1.193150814374288 |
  
---
  
We can extract the following findings:
- The results are very good, both in terms of performance and in terms of execution time.  
  
- While increasing the dimensionality of the problem does lead to worse solutions overall, the decrease in fitness quality is not drastic, while the increase in execution time is almost negligible.

## IWOA Experiments

---

**Functions**

In [ ]:
def iwoa_n_times(n, pop_size, nr_dim, func, bounds, nr_iter, F, CR):
  """
    Runs the Improved Whale Optimization Algorithm n times and returns the obtained results as a list
  """
  best_solutions = []
  for step in range(n):
    _, best_fitness = iwoa(pop_size=pop_size, nr_dim=nr_dim, func=func, bounds=bounds, nr_iter=nr_iter, F=F, CR=CR)
    best_solutions.append(best_fitness)
  return best_solutions

---

**IWOA ideal parameters**  

We have several parameters which affect the performance of the Improved Whale Optimization algorithm:  
- Population size: `pop_size`
- Number of iterations: `nr_iter`
- Differential weight: `F`
- Crossover probability: `CR`

Following the works of *Seyed Mostafa Bozorgi* and *Samaneh Yazdani*, let us consider their proposed parameters:
```python
  pop_size = 30
  nr_iter = 500
  F = 0.5
  CR = 0.9
```
and tweak these values to see if we can determine a better performing configuration.  

For the sake of minimizing computational resources, the following comparative analysis will be performed on all three functions, considered on a low dimensionality setting:
```python
  func in [schwefel, ackley, griewank]
  nr_dim = 2
```

In [ ]:
# Test the algorithm for different values for constants

func_values = [schwefel, ackley, griewank]
bounds_values = [(-100, 100), (-32, 32), (-600, 600)]

pop_size_values = [20, 30, 50]
nr_iter_values = [100, 500]
F_values = [0.3, 0.5, 0.7]
CR_values = [0.5, 0.9]
# total expermients: 3 * 3 * 2 * 3 * 2 = 108

func_list = []
pop_size_list = []
nr_iter_list = []
F_list = []
CR_list = []

fitness_list = []
execution_time_list = []
for func, bounds in zip(func_values, bounds_values):
  for pop_size in pop_size_values:
    for nr_iter in nr_iter_values:
      for F in F_values:
        for CR in CR_values:
          # perform 30 identical experiments
          start_time = time.time()
          best_fitnesses = iwoa_n_times(30, pop_size=pop_size, nr_dim=2, func=func, bounds=bounds, nr_iter=nr_iter, F=F, CR=CR)
          end_time = time.time()

          average_fitness = np.array(best_fitnesses).mean()
          average_execution_time = (end_time - start_time) / 30

          func_list.append(func.__name__)
          pop_size_list.append(pop_size)
          nr_iter_list.append(nr_iter)
          F_list.append(F)
          CR_list.append(CR)

          fitness_list.append(average_fitness)
          execution_time_list.append(average_execution_time)

# Display experiment results in markdown table
header = "| Function | Population size | Maximum iterations number | Differential weight | Crossover probability | Average fitness | Average execution time |\n"
header += "|---|---|---|---|---|---|---|\n"
markdown_table_4 = lists_to_markdown_table(header, func_list, pop_size_list, nr_iter_list, F_list, CR_list, fitness_list, execution_time_list)

---

**Schwefel**  

| Function | Population size | Maximum iterations number | Differential weight | Crossover probability | Average fitness | Average execution time |
|---|---|---|---|---|---|---|
| schwefel | 20 | 100 | 0.3 | 0.5 | 9.811062353465244e-19 | 0.19429778258005778 |
| schwefel | 20 | 100 | 0.3 | 0.9 | 7.198098762682884e-16 | 0.16335589090983074 |
| schwefel | 20 | 100 | 0.5 | 0.5 | 3.777413790735346e-18 | 0.1662188450495402 |
| schwefel | 20 | 100 | 0.5 | 0.9 | 2.9419718307434956e-17 | 0.1634324868520101 |
| schwefel | 20 | 100 | 0.7 | 0.5 | 7.119017425206223e-14 | 0.16991766293843588 |
| schwefel | 20 | 100 | 0.7 | 0.9 | 1.0740383150895977e-13 | 0.16587496598561605 |
| schwefel | 20 | 500 | 0.3 | 0.5 | 2.4559696860367603e-106 | 0.8183659791946412 |
| schwefel | 20 | 500 | 0.3 | 0.9 | 5.617856491701147e-88 | 0.795008905728658 |
| schwefel | 20 | 500 | 0.5 | 0.5 | 5.474909014322196e-88 | 0.8388508876164754 |
| schwefel | 20 | 500 | 0.5 | 0.9 | 3.789164193776193e-91 | 0.7989426612854004 |
| schwefel | 20 | 500 | 0.7 | 0.5 | 8.460182597653035e-77 | 0.8404430468877157 |
| schwefel | 20 | 500 | 0.7 | 0.9 | 1.2064697974944632e-73 | 0.8122621218363444 |
| schwefel | 30 | 100 | 0.3 | 0.5 | 1.1057498030113398e-25 | 0.26446491877237954 |
| schwefel | 30 | 100 | 0.3 | 0.9 | 4.221892903377373e-28 | 0.24931968053181966 |
| schwefel | 30 | 100 | 0.5 | 0.5 | 1.461575499655791e-20 | 0.25526496569315593 |
| schwefel | 30 | 100 | 0.5 | 0.9 | 1.9888413727949947e-21 | 0.2428597609202067 |
| schwefel | 30 | 100 | 0.7 | 0.5 | 9.140670376720347e-18 | 0.2595715920130412 |
| schwefel | 30 | 100 | 0.7 | 0.9 | 1.6745438912900137e-17 | 0.24440619945526124 |
| schwefel | 30 | 500 | 0.3 | 0.5 | 5.092796441197738e-124 | 1.2845318078994752 |
| schwefel | 30 | 500 | 0.3 | 0.9 | 9.260326849160985e-132 | 1.257014012336731 |
| schwefel | 30 | 500 | 0.5 | 0.5 | 1.816550776002693e-103 | 1.3059231281280517 |
| schwefel | 30 | 500 | 0.5 | 0.9 | 3.167982505323433e-112 | 1.2214995702107747 |
| schwefel | 30 | 500 | 0.7 | 0.5 | 2.9370937488659846e-91 | 1.2738443613052368 |
| schwefel | 30 | 500 | 0.7 | 0.9 | 3.990818505541334e-91 | 1.257186730702718 |
| schwefel | 50 | 100 | 0.3 | 0.5 | 1.2574746199239616e-29 | 0.44294604460398357 |
| schwefel | 50 | 100 | 0.3 | 0.9 | 3.790306717809034e-34 | 0.4391590754191081 |
| schwefel | 50 | 100 | 0.5 | 0.5 | 3.252694291177452e-24 | 0.453536057472229 |
| schwefel | 50 | 100 | 0.5 | 0.9 | 1.9357075908317756e-27 | 0.4341862281163534 |
| schwefel | 50 | 100 | 0.7 | 0.5 | 6.497308091025862e-21 | 0.4518094778060913 |
| schwefel | 50 | 100 | 0.7 | 0.9 | 2.4128085034803975e-21 | 0.4224781592686971 |
| schwefel | 50 | 500 | 0.3 | 0.5 | 1.059547619851474e-148 | 2.18649259408315 |
| schwefel | 50 | 500 | 0.3 | 0.9 | 2.3662808104010188e-169 | 2.1143165111541746 |
| schwefel | 50 | 500 | 0.5 | 0.5 | 2.8899136149423623e-124 | 2.1612863222757976 |
| schwefel | 50 | 500 | 0.5 | 0.9 | 2.3678475014871276e-130 | 2.1257582823435466 |
| schwefel | 50 | 500 | 0.7 | 0.5 | 7.710205005929975e-106 | 2.1754753748575846 |
| schwefel | 50 | 500 | 0.7 | 0.9 | 7.851992541046236e-109 | 2.1202921708424887 |

---

**Ackley**  

| Function | Population size | Maximum iterations number | Differential weight | Crossover probability | Average fitness | Average execution time |
|---|---|---|---|---|---|---|
| ackley | 20 | 100 | 0.3 | 0.5 | 1.2730557349035128e-15 | 0.20073615709940593 |
| ackley | 20 | 100 | 0.3 | 0.9 | 1.7467508920769129e-15 | 0.19634302457173666 |
| ackley | 20 | 100 | 0.5 | 0.5 | 1.2730557349035128e-15 | 0.19471267064412434 |
| ackley | 20 | 100 | 0.5 | 0.9 | 1.3914795241968629e-15 | 0.19444809754689535 |
| ackley | 20 | 100 | 0.7 | 0.5 | 1.865174681370263e-15 | 0.20143388907114665 |
| ackley | 20 | 100 | 0.7 | 0.9 | 2.220446049250313e-15 | 0.19511493047078451 |
| ackley | 20 | 500 | 0.3 | 0.5 | 6.809367884367627e-16 | 0.990899920463562 |
| ackley | 20 | 500 | 0.3 | 0.9 | 5.625129991434126e-16 | 0.951150902112325 |
| ackley | 20 | 500 | 0.5 | 0.5 | 4.440892098500626e-16 | 1.000394622484843 |
| ackley | 20 | 500 | 0.5 | 0.9 | 4.440892098500626e-16 | 0.9732355117797852 |
| ackley | 20 | 500 | 0.7 | 0.5 | 4.440892098500626e-16 | 1.0001524766286214 |
| ackley | 20 | 500 | 0.7 | 0.9 | 4.440892098500626e-16 | 0.9650051991144816 |
| ackley | 30 | 100 | 0.3 | 0.5 | 1.0362081563168128e-15 | 0.3014802535374959 |
| ackley | 30 | 100 | 0.3 | 0.9 | 1.1546319456101628e-15 | 0.2957592248916626 |
| ackley | 30 | 100 | 0.5 | 0.5 | 7.993605777301127e-16 | 0.30368101596832275 |
| ackley | 30 | 100 | 0.5 | 0.9 | 1.3914795241968629e-15 | 0.30486316680908204 |
| ackley | 30 | 100 | 0.7 | 0.5 | 1.5099033134902129e-15 | 0.30153798262278236 |
| ackley | 30 | 100 | 0.7 | 0.9 | 1.1546319456101628e-15 | 0.305311393737793 |
| ackley | 30 | 500 | 0.3 | 0.5 | 4.440892098500626e-16 | 1.5193537553151448 |
| ackley | 30 | 500 | 0.3 | 0.9 | 5.625129991434126e-16 | 1.4534314076105754 |
| ackley | 30 | 500 | 0.5 | 0.5 | 4.440892098500626e-16 | 1.514247751235962 |
| ackley | 30 | 500 | 0.5 | 0.9 | 4.440892098500626e-16 | 1.4551809787750245 |
| ackley | 30 | 500 | 0.7 | 0.5 | 4.440892098500626e-16 | 1.5167196909586589 |
| ackley | 30 | 500 | 0.7 | 0.9 | 4.440892098500626e-16 | 1.486431360244751 |
| ackley | 50 | 100 | 0.3 | 0.5 | 4.440892098500626e-16 | 0.531304677327474 |
| ackley | 50 | 100 | 0.3 | 0.9 | 7.993605777301127e-16 | 0.5042625347773234 |
| ackley | 50 | 100 | 0.5 | 0.5 | 6.809367884367627e-16 | 0.5130382140477499 |
| ackley | 50 | 100 | 0.5 | 0.9 | 6.809367884367627e-16 | 0.5113192478815715 |
| ackley | 50 | 100 | 0.7 | 0.5 | 4.440892098500626e-16 | 0.530804451306661 |
| ackley | 50 | 100 | 0.7 | 0.9 | 6.809367884367627e-16 | 0.5024725914001464 |
| ackley | 50 | 500 | 0.3 | 0.5 | 4.440892098500626e-16 | 2.58433043162028 |
| ackley | 50 | 500 | 0.3 | 0.9 | 9.177843670234628e-16 | 2.544195032119751 |
| ackley | 50 | 500 | 0.5 | 0.5 | 4.440892098500626e-16 | 2.6104299068450927 |
| ackley | 50 | 500 | 0.5 | 0.9 | 4.440892098500626e-16 | 2.5328736384709676 |
| ackley | 50 | 500 | 0.7 | 0.5 | 4.440892098500626e-16 | 2.6328505198160808 |
| ackley | 50 | 500 | 0.7 | 0.9 | 4.440892098500626e-16 | 2.565674146016439 |

---

**Griewank**  

| Function | Population size | Maximum iterations number | Differential weight | Crossover probability | Average fitness | Average execution time |
|---|---|---|---|---|---|---|
| griewank | 20 | 100 | 0.3 | 0.5 | 0.01199955530339345 | 0.1922460953394572 |
| griewank | 20 | 100 | 0.3 | 0.9 | 0.02769942727060954 | 0.18625116348266602 |
| griewank | 20 | 100 | 0.5 | 0.5 | 0.0027941691800744796 | 0.19264017740885417 |
| griewank | 20 | 100 | 0.5 | 0.9 | 0.004355664970215574 | 0.18962031205495197 |
| griewank | 20 | 100 | 0.7 | 0.5 | 0.009369686061753904 | 0.19976926644643148 |
| griewank | 20 | 100 | 0.7 | 0.9 | 0.011177990682912838 | 0.190375550587972 |
| griewank | 20 | 500 | 0.3 | 0.5 | 0.002465346778038328 | 0.9690429528554281 |
| griewank | 20 | 500 | 0.3 | 0.9 | 0.0021365243753530844 | 0.9631063779195149 |
| griewank | 20 | 500 | 0.5 | 0.5 | 0.0004930693556076597 | 0.9564532915751139 |
| griewank | 20 | 500 | 0.5 | 0.9 | 0.002383059052899631 | 0.929554812113444 |
| griewank | 20 | 500 | 0.7 | 0.5 | 0.003204950811449788 | 0.9687867164611816 |
| griewank | 20 | 500 | 0.7 | 0.9 | 0.0004930693556076597 | 0.9252235730489095 |
| griewank | 30 | 100 | 0.3 | 0.5 | 0.0073966973169081515 | 0.28032503128051756 |
| griewank | 30 | 100 | 0.3 | 0.9 | 0.016930575810255203 | 0.27872642676035564 |
| griewank | 30 | 100 | 0.5 | 0.5 | 0.0004930693556076634 | 0.2881105899810791 |
| griewank | 30 | 100 | 0.5 | 0.9 | 0.005588995343019717 | 0.2829772392908732 |
| griewank | 30 | 100 | 0.7 | 0.5 | 0.005588666852378776 | 0.2909715096155802 |
| griewank | 30 | 100 | 0.7 | 0.9 | 0.005670954576325996 | 0.2905012289683024 |
| griewank | 30 | 500 | 0.3 | 0.5 | 0.0024653467780382985 | 1.454866393407186 |
| griewank | 30 | 500 | 0.3 | 0.9 | 0.0012326733890191492 | 1.3979824225107829 |
| griewank | 30 | 500 | 0.5 | 0.5 | 0.0009861387112153194 | 1.4362513780593873 |
| griewank | 30 | 500 | 0.5 | 0.9 | 0.0007396040334114895 | 1.4119396209716797 |
| griewank | 30 | 500 | 0.7 | 0.5 | 0.001479208066822979 | 1.4185717105865479 |
| griewank | 30 | 500 | 0.7 | 0.9 | 0.0012326733890191492 | 1.3968763510386148 |
| griewank | 50 | 100 | 0.3 | 0.5 | 0.00386226711881366 | 0.4990886370340983 |
| griewank | 50 | 100 | 0.3 | 0.9 | 0.0040271710610435925 | 0.47289822101593015 |
| griewank | 50 | 100 | 0.5 | 0.5 | 0.0007396040334114932 | 0.4942557573318481 |
| griewank | 50 | 100 | 0.5 | 0.9 | 0.0010684264354450246 | 0.485453200340271 |
| griewank | 50 | 100 | 0.7 | 0.5 | 0.004437624199560012 | 0.5001458247502645 |
| griewank | 50 | 100 | 0.7 | 0.9 | 0.00599977697633581 | 0.48581523895263673 |
| griewank | 50 | 500 | 0.3 | 0.5 | 0.001725742744626809 | 2.4501224915186564 |
| griewank | 50 | 500 | 0.3 | 0.9 | 0.001479208066822979 | 2.359857130050659 |
| griewank | 50 | 500 | 0.5 | 0.5 | 0.0 | 2.4620445092519123 |
| griewank | 50 | 500 | 0.5 | 0.9 | 0.0004930693556076597 | 2.374594982465108 |
| griewank | 50 | 500 | 0.7 | 0.5 | 0.0024653467780382985 | 2.5272974252700804 |
| griewank | 50 | 500 | 0.7 | 0.9 | 0.0012326733890191492 | 2.3832698504130048 |

---
**Analyzing individual parameter performance**

In [ ]:
# Analyse fitness by each parameter option

pop_size_values = [20, 30, 50]
nr_iter_values = [100, 500]
F_values = [0.3, 0.5, 0.7]
CR_values = [0.5, 0.9]

# average fitness and average time by pop_size
print("Average fitness and average time by population size:")
for pop_size in pop_size_values:
  fitnesses = []
  execution_times = []
  for i in range(len(fitness_list)):
    if pop_size_list[i] == pop_size:
      fitnesses.append(fitness_list[i])
      execution_times.append(execution_time_list[i])
  average_fitness = np.array(fitnesses).mean()
  average_execution_time = np.array(execution_times).mean()
  # print results
  print(f"pop_size value: {pop_size}, average fitness: {average_fitness}, average execution time: {average_execution_time}")
print()

# average fitness and average time by nr_iter
print("Average fitness and average time by number of iterations:")
for nr_iter in nr_iter_values:
  fitnesses = []
  execution_times = []
  for i in range(len(fitness_list)):
    if nr_iter_list[i] == nr_iter:
      fitnesses.append(fitness_list[i])
      execution_times.append(execution_time_list[i])
  average_fitness = np.array(fitnesses).mean()
  average_execution_time = np.array(execution_times).mean()
  # print results
  print(f"nr_iter value: {nr_iter}, average fitness: {average_fitness}, average execution time: {average_execution_time}")
print()

# average fitness and average time by F
print("Average fitness and average time by differential weight:")
for F in F_values:
  fitnesses = []
  execution_times = []
  for i in range(len(fitness_list)):
    if F_list[i] == F:
      fitnesses.append(fitness_list[i])
      execution_times.append(execution_time_list[i])
  average_fitness = np.array(fitnesses).mean()
  average_execution_time = np.array(execution_times).mean()
  # print results
  print(f"F value: {F}, average fitness: {average_fitness}, average execution time: {average_execution_time}")
print()

# average fitness and average time by CR
print("Average fitness and average time by crossover probability:")
for CR in CR_values:
  fitnesses = []
  execution_times = []
  for i in range(len(fitness_list)):
    if CR_list[i] == CR:
      fitnesses.append(fitness_list[i])
      execution_times.append(execution_time_list[i])
  average_fitness = np.array(fitnesses).mean()
  average_execution_time = np.array(execution_times).mean()
  # print results
  print(f"CR value: {CR}, average fitness: {average_fitness}, average execution time: {average_execution_time}")


Average fitness and average time by population size:
pop_size value: 20, average fitness: 0.0021825698110585576, average execution time: 0.5514908035596212
pop_size value: 30, average fitness: 0.0013834612117231042, average execution time: 0.836166239226306
pop_size value: 50, average fitness: 0.0007647475044092045, average execution time: 1.4329481932852002

Average fitness and average time by number of iterations:
nr_iter value: 100, average fitness: 0.0023925985657086072, average execution time: 0.31614619979151976
nr_iter value: 500, average fitness: 0.0004945871190853031, average execution time: 1.564257290922565

Average fitness and average time by differential weight:
F value: 0.3, average fitness: 0.002317237111470362, average execution time: 0.9385287185510001
F value: 0.5, average fitness: 0.000559301939625684, average execution time: 0.9383355524804856
F value: 0.7, average fitness: 0.00145423947609482, average execution time: 0.9437409650396419

Average fitness and average 

We see that, on average, the best values for the three parameters are the following:
```python
  pop_size = 50
  nr_iter = 100
  F = 0.5
  CR = 0.5
```
Now, let us extract the 10 actual best performing configurations in terms of average fitness.

---

**Top 10 best performing configurations**

In [ ]:
# Extract and print best 10 configurations

average_fitnesses_list = []
average_execution_times_list = []
values_no = (int) (len(fitness_list) / 3)

for i in range(values_no):
  average_fitness = (fitness_list[i] + fitness_list[values_no + i] + fitness_list[2 * values_no + i] ) / 3
  average_execution_time = (execution_time_list[i] + execution_time_list[values_no + i] + execution_time_list[2 * values_no + i] ) / 3
  average_fitnesses_list.append(average_fitness)
  average_execution_times_list.append(average_execution_time)

best_pop_size_list = []
best_nr_iter_list = []
best_F_list = []
best_CR_list = []

best_average_fitnesses_list = []
best_average_execution_times_list = []

best_indices = [average_fitnesses_list.index(x) for x in sorted(average_fitnesses_list, reverse=True)[-10:]]
best_indices.reverse()

for i in best_indices:
  best_pop_size_list.append(pop_size_list[i])
  best_nr_iter_list.append(nr_iter_list[i])
  best_F_list.append(F_list[i])
  best_CR_list.append(CR_list[i])
  best_average_fitnesses_list.append(average_fitnesses_list[i])
  best_average_execution_times_list.append(average_execution_times_list[i])

# Display experiment results in markdown table
header = "| Function | Population size | Maximum iterations number | Differential weight | Crossover probability | Average fitness | Average execution time |\n"
header += "|---|---|---|---|---|---|---|\n"
markdown_table_5 = lists_to_markdown_table(header, list(range(1, 11)), best_pop_size_list, best_nr_iter_list, best_F_list, best_CR_list, best_average_fitnesses_list, best_average_execution_times_list)

| Function | Population size | Maximum iterations number | Differential weight | Crossover probability | Average fitness | Average execution time |
|---|---|---|---|---|---|---|
| 1 | 50 | 500 | 0.5 | 0.5 | 8.217822593475797e-05 | 2.3724822945064967 |
| 2 | 50 | 100 | 0.5 | 0.5 | 0.00016435645186944689 | 0.48331989977094864 |
| 3 | 50 | 100 | 0.5 | 0.9 | 0.0002465346778039779 | 0.4687444421980116 |
| 4 | 30 | 100 | 0.5 | 0.5 | 0.00035614214514864884 | 0.27165558603074813 |
| 5 | 50 | 500 | 0.5 | 0.9 | 0.0004108911296731978 | 2.3394191424051916 |
| 6 | 30 | 500 | 0.5 | 0.5 | 0.0004383203710830995 | 1.392393636703491 |
| 7 | 30 | 100 | 0.5 | 0.9 | 0.0010411066916558352 | 0.27038318845960835 |
| 8 | 50 | 100 | 0.7 | 0.5 | 0.0012988049116601076 | 0.46460626125335686 |
| 9 | 20 | 500 | 0.5 | 0.5 | 0.001451778816817294 | 0.9139759063720704 |
| 10 | 30 | 100 | 0.7 | 0.5 | 0.0018041602473334586 | 0.273529224925571 |  
  
We can extract the following observations:

- Unlike in the previous scenario, where performance was mainly based on the `pop_size` and `nr_iter`, we now see a higher variety of viable configurations, yielding great results.

- The main constraint can come in terms of computational resources, where a higher number of iterations significantly increases the execution time.

- We observe that the configuration proposed by *Seyed Mostafa Bozorgi* and *Samaneh Yazdani* is fifth in our list. It seems to be significantly outperformed by configuration *1*, with a reduced crossover probability. This makes sense in the context of our simpler functions, where exploitation is preffered over exploration for the most part.

- Depending on computational resources available and desired accuracy, one can choose one of the following two configurations:
```python
  # Higher execution time, best fitness
  pop_size, nr_iter, F, CR = 50, 500, 0.5, 0.5
  # Lower execution time, good fitness
  pop_size, nr_iter, spiral_shape_const = 50, 100, 0.5, 0.5
```

---

**Performance based on function dimensions**

In [ ]:
# Test the algorithm for different function dimensions

func_values = [schwefel, ackley, griewank]
bounds_values = [(-100, 100), (-32, 32), (-600, 600)]

nr_dim_values = [2, 5, 10, 20, 30]
# total expermients: 3 * 5 = 15

func_list = []
nr_dim_list = []

fitness_list = []
std_dev_list = []
execution_time_list = []

pop_size, nr_iter, F, CR = 50, 500, 0.5, 0.5

for func, bounds in zip(func_values, bounds_values):
  for nr_dim in nr_dim_values:
    # perform 30 identical experiments
    start_time = time.time()
    best_fitnesses = iwoa_n_times(30, pop_size=pop_size, nr_dim=nr_dim, func=func, bounds=bounds, nr_iter=nr_iter, F=F, CR=CR)
    end_time = time.time()

    average_fitness = np.array(best_fitnesses).mean()
    average_execution_time = (end_time - start_time) / 30

    func_list.append(func.__name__)
    nr_dim_list.append(nr_dim)

    fitness_list.append(average_fitness)
    execution_time_list.append(average_execution_time)

# Display experiment results in markdown table
header = "| Function | Number of dimensions | Average fitness | Average execution time |\n"
header += "|---|---|---|---|\n"
markdown_table_6 = lists_to_markdown_table(header, func_list, nr_dim_list, fitness_list, execution_time_list)

---

**Schwefel**  

| Function | Number of dimensions | Average fitness | Average execution time |
|---|---|---|---|
| schwefel | 2 | 1.4465968755643116e-119 | 0.7727345148722331 |
| schwefel | 5 | 3.3404708867691244e-43 | 0.9534897168477376 |
| schwefel | 10 | 3.880295288119376e-14 | 1.3597123781840006 |
| schwefel | 20 | 0.00028204712425731267 | 2.846091850598653 |
| schwefel | 30 | 0.1316886455459488 | 4.8049023151397705 |

---

**Ackley**  

| Function | Number of dimensions | Average fitness | Average execution time |
|---|---|---|---|
| ackley | 2 | 4.440892098500626e-16 | 1.2476124922434488 |
| ackley | 5 | 1.7467508920769129e-15 | 1.354746397336324 |
| ackley | 10 | 3.9968028886505635e-15 | 1.8403050661087037 |
| ackley | 20 | 6.720550042397614e-15 | 4.6829907973607385 |
| ackley | 30 | 9.681144774731365e-15 | 3.2949053764343263 |

---

**Griewank**  

| Function | Number of dimensions | Average fitness | Average execution time |
|---|---|---|---|
| griewank | 2 | 0.0 | 1.214130735397339 |
| griewank | 5 | 0.007963607130222634 | 1.8366588672002158 |
| griewank | 10 | 0.02870218268017842 | 3.3393203020095825 |
| griewank | 20 | 0.009763462758916482 | 3.156192485491435 |
| griewank | 30 | 0.0017239399212942337 | 9.4741979042689 |
---
  
We can extract the following findings:
- The improved algorithm yields great results for both low and high dimensionality problems, at the cost of significantly increased execution time.  

- It outperforms the simple WOA for all experiments, and proves that hibridization is the key in solving the problem at hand.

- The current parameter settings, however, put high emphasis on refining solutions and exploitation, given the medium value for the crossover rate.  

Our last batch of experiments will test the same setting, this time with a higher value of `CR = 0.9` to search the space even more and hopefully achieve even better results on higher dimensionality settings.

In [ ]:
# Test the algorithm for different function dimensions

func_values = [schwefel, ackley, griewank]
bounds_values = [(-100, 100), (-32, 32), (-600, 600)]

nr_dim_values = [2, 5, 10, 20, 30]
# total expermients: 3 * 5 = 15

func_list = []
nr_dim_list = []

fitness_list = []
std_dev_list = []
execution_time_list = []

pop_size, nr_iter, F, CR = 50, 500, 0.5, 0.5

for func, bounds in zip(func_values, bounds_values):
  for nr_dim in nr_dim_values:
    # perform 30 identical experiments
    start_time = time.time()
    best_fitnesses = iwoa_n_times(5, pop_size=pop_size, nr_dim=nr_dim, func=func, bounds=bounds, nr_iter=nr_iter, F=F, CR=CR)
    end_time = time.time()

    average_fitness = np.array(best_fitnesses).mean()
    average_execution_time = (end_time - start_time) / 30

    func_list.append(func.__name__)
    nr_dim_list.append(nr_dim)

    fitness_list.append(average_fitness)
    execution_time_list.append(average_execution_time)


In [ ]:
# Display experiment results in markdown table
header = "| Function | Number of dimensions | Average fitness | Average execution time |\n"
header += "|---|---|---|---|\n"
markdown_table_6 = lists_to_markdown_table(header, func_list, nr_dim_list, fitness_list, execution_time_list)

---

**Schwefel**  

| Function | Number of dimensions | Average fitness | Average execution time |
|---|---|---|---|
| schwefel | 2 | 1.6394124737951584e-136 | 1.0362447102864583 |
| schwefel | 5 | 8.702871176078456e-57 | 5.012729128201802 |
| schwefel | 10 | 1.6477330026742203e-16 | 1.0849886337916057 |
| schwefel | 20 | 0.00016693353998139136 | 1.873312520980835 |
| schwefel | 30 | 0.03358727988046083 | 2.6538828214009604 |

---

**Ackley**  

| Function | Number of dimensions | Average fitness | Average execution time |
|---|---|---|---|
| ackley | 2 | 4.440892098500626e-16 | 1.3849728186925252 |
| ackley | 5 | 2.575717417130363e-15 | 1.579448930422465 |
| ackley | 10 | 4.588921835117314e-15 | 2.0053362369537355 |
| ackley | 20 | 8.733754460384565e-15 | 3.412826085090637 |
| ackley | 30 | 8.141635513917814e-15 | 5.0647656440734865 |

---

**Griewank**  

| Function | Number of dimensions | Average fitness | Average execution time |
|---|---|---|---|
| griewank | 2 | 0.0 | 1.3547224601109822 |
| griewank | 5 | 0.0076361781589597945 | 1.533016562461853 |
| griewank | 10 | 0.03205287969122984 | 1.8080792744954428 |
| griewank | 20 | 0.005002661961238719 | 2.392739748954773 |
| griewank | 30 | 0.0029555468640962367 | 2.914011311531067 |

We can extract the following observations:
- By increasing the crossover rate, we managed to improve the best solution for two out of the three functions considered, namely *Schwefel* and *Ackley*, while we got slightly worse results for *Griewank*.

- We also managed to vastly reduce the execution time, compared to the last batch of experiments.

- This proves that the ideal settings for the IWOA might be function and dimensionality dependant.  

As a final conslusion, we managed to reach very good results for the optimization problems considered, within reasonable execution time, using the Improved Whale Optimization Algorithm designed.